#  News Content Collect and Store
## by Aly Abdelrazek
 
 


##Importing libraries

In [6]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pickle


##Global variables

In [12]:
news_page_url = "https://bbc.com/news"
absolute_link_prefix = "https://bbc.com"


##Scraping


In [14]:
# Scrapes news stories from news_page_url (https://bbc.com/news)
def news_page_to_news_urls(url, prefix):
    ''' Takes the URL to a news page and the prefix of the absolute link, and returns a list of news stories URLs.'''
    news_urls = []
    news_page_html = requests.get(url).text
    news_page_soup = BeautifulSoup(news_page_html, "html.parser")
    news_page_links = news_page_soup.findAll("a") #find all links
    for link in news_page_links:
        href = link.get("href")
        if href.startswith("/news") and href[-1].isdigit():
           news_url = prefix + href
           news_urls.append(news_url)
    return news_urls

news_urls = news_page_to_news_urls(news_page_url, absolute_link_prefix)